<a href="https://colab.research.google.com/github/sakuna47/AI_Maze/blob/Code/AI_MAZE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random  # For random selection of start, goal, and barriers
import math  # For calculating diagonal costs (sqrt(2))
import heapq  # For priority queue in UCS and A* search
import statistics  # For calculating mean and variance in analysis
from collections import deque  # For BFS in solvability check

In [ ]:
# Convert a node number (0-35) to (x, y) coordinates in a 6x6 grid
# x = node % 6 (column), y = node // 6 (row)
# Example: Node 15 -> (x=2, y=3)
def get_coordinates(node):
    return (node % 6, node // 6)


In [ ]:
# Get valid neighbors of a node with their edge costs
# Neighbors are in 8 directions: horizontal, vertical, diagonal
# Barriers block movement; neighbors are sorted by node number for deterministic exploration
def get_neighbors(node, barriers):
    x, y = get_coordinates(node)  # Get current node's coordinates
    # Define 8 possible directions: up, down, left, right, and diagonals
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]
    neighbors = []
    for dx, dy in directions:
        new_x, new_y = x + dx, y + dy  # Calculate neighbor's coordinates
        # Check if the neighbor is within the 6x6 grid
        if 0 <= new_x < 6 and 0 <= new_y < 6:
            neighbor = new_y * 6 + new_x  # Convert coordinates back to node number
            if neighbor not in barriers:  # Exclude barriers
                # Cost is 1 for horizontal/vertical moves, sqrt(2) for diagonal moves
                cost = 1 if dx == 0 or dy == 0 else math.sqrt(2)
                neighbors.append((neighbor, cost))
    # Sort neighbors by node number to ensure consistent exploration order
    # Example: For node 8, neighbors should be processed as 2, 7, 9, 14
    neighbors.sort(key=lambda x: x[0])
    return neighbors

In [ ]:

# Use BFS to check if a path exists between start and goal nodes
# Ensures the maze is solvable before proceeding with search algorithms
def is_solvable(start, goal, barriers):
    visited = set()  # Track visited nodes
    queue = deque([start])  # Queue for BFS
    visited.add(start)
    while queue:
        node = queue.popleft()  # Process the next node
        if node == goal:  # Path found
            return True
        # Explore neighbors of the current node
        for neighbor, _ in get_neighbors(node, barriers):
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)
    return False  # No path found

In [ ]:
# Task 1: Set up a 6x6 maze with random start, goal, and barriers
# Start node: 0-11, Goal node: 24-35, Barriers: 4 random nodes
# Ensures the maze is solvable using BFS
def setup_maze():
    while True:
        start = random.randint(0, 11)  # Random start node from 0-11
        goal = random.randint(24, 35)  # Random goal node from 24-35
        # Get remaining nodes (excluding start and goal) for barrier selection
        remaining = [i for i in range(36) if i != start and i != goal]
        barriers = random.sample(remaining, 4)  # Select 4 barriers
        # Check if a path exists from start to goal
        if is_solvable(start, goal, barriers):
            return start, goal, barriers

In [ ]:
# Task 2: Uniform Cost Search (UCS) to find the shortest path
# Uses a priority queue to explore nodes by lowest path cost
# Outputs visited nodes, time (nodes explored), and final path
def uniform_cost_search(start, goal, barriers):
    # Priority queue: (cost, node, path)
    queue = [(0, start, [start])]
    visited = set()  # Track visited nodes
    while queue:
        cost, node, path = heapq.heappop(queue)  # Get node with lowest cost
        if node in visited:  # Skip if already visited
            continue
        visited.add(node)  # Mark node as visited
        if node == goal:  # Goal reached
            return list(visited), len(visited), path
        # Explore neighbors of the current node
        for neighbor, edge_cost in get_neighbors(node, barriers):
            if neighbor not in visited:
                new_cost = cost + edge_cost  # Update path cost
                new_path = path + [neighbor]  # Extend path
                heapq.heappush(queue, (new_cost, neighbor, new_path))
    # No path found (shouldn't happen due to solvability check)
    return list(visited), len(visited), []

In [ ]:

# Task 3: Calculate Chebyshev Distance heuristic for A* search
# Formula: max(|Nx - Gx|, |Ny - Gy|)
# Used to estimate the remaining distance to the goal
def chebyshev_distance(node, goal):
    x1, y1 = get_coordinates(node)  # Current node's coordinates
    x2, y2 = get_coordinates(goal)  # Goal node's coordinates
    return max(abs(x1 - x2), abs(y1 - y2))  # Chebyshev Distance

In [22]:
# Task 4: A* Search to find the shortest path
# Combines path cost (g) and heuristic (h) to guide the search
# Outputs visited nodes, time (nodes explored), and final path
def a_star_search(start, goal, barriers):
    # Priority queue: (f, g, node, path), where f = g + h
    queue = [(chebyshev_distance(start, goal), 0, start, [start])]
    visited = set()  # Track visited nodes
    while queue:
        _, g_cost, node, path = heapq.heappop(queue)  # Get node with lowest f
        if node in visited:  # Skip if already visited
            continue
        visited.add(node)  # Mark node as visited
        if node == goal:  # Goal reached
            return list(visited), len(visited), path
        # Explore neighbors of the current node
        for neighbor, edge_cost in get_neighbors(node, barriers):
            if neighbor not in visited:
                new_g = g_cost + edge_cost  # Update path cost (g)
                h = chebyshev_distance(neighbor, goal)  # Heuristic cost (h)
                new_path = path + [neighbor]  # Extend path
                heapq.heappush(queue, (new_g + h, new_g, neighbor, new_path))
    # No path found (shouldn't happen due to solvability check)
    return list(visited), len(visited), []

In [20]:
# Visualize the maze with node numbers, barriers, start, goal, and path
# Not required by coursework but useful for debugging and presentation
def visualize_maze(start, goal, barriers, path):
    # Initialize a 6x6 grid with node numbers
    grid = [['.' for _ in range(6)] for _ in range(6)]
    for node in range(36):
        x, y = get_coordinates(node)
        grid[y][x] = str(node).zfill(2)  # Format node number (e.g., "08")
    # Mark barriers
    for b in barriers:
        x, y = get_coordinates(b)
        grid[y][x] = 'XX'  # Represent barriers
    # Mark start and goal
    x, y = get_coordinates(start)
    grid[y][x] = 'SS'  # Start node
    x, y = get_coordinates(goal)
    grid[y][x] = 'GG'  # Goal node
    # Mark the path (excluding start and goal)
    for node in path[1:-1]:
        x, y = get_coordinates(node)
        grid[y][x] = '**'  # Path nodes
    # Print the grid
    for row in grid:
        print(' '.join(row))

In [19]:
# Task 5: Run experiments on three random mazes
# For each maze, perform UCS and A* search, and collect results
def run_experiments():
    results = []  # Store results for analysis
    for i in range(3):
        print(f"\nMaze {i+1}")
        # Set up a new random maze
        start, goal, barriers = setup_maze()
        print(f"Start: {start}, Goal: {goal}, Barriers: {barriers}")

        # Perform UCS
        ucs_visited, ucs_time, ucs_path = uniform_cost_search(start, goal, barriers)
        print(f"UCS - Visited: {sorted(ucs_visited)}, Time: {ucs_time} mins, Path: {ucs_path}")
        print("UCS Visualization:")
        visualize_maze(start, goal, barriers, ucs_path)

        # Perform A*
        astar_visited, astar_time, astar_path = a_star_search(start, goal, barriers)
        print(f"A* - Visited: {sorted(astar_visited)}, Time: {astar_time} mins, Path: {astar_path}")
        print("A* Visualization:")
        visualize_maze(start, goal, barriers, astar_path)

        # Store metrics for analysis
        results.append({
            'ucs_time': ucs_time,
            'ucs_path_len': len(ucs_path) - 1 if ucs_path else 0,  # Path length excludes start
            'astar_time': astar_time,
            'astar_path_len': len(astar_path) - 1 if astar_path else 0
        })
    return results

In [23]:
# Analyze the performance of UCS and A* across the three mazes
# Compute mean and variance of time and path length, and provide theoretical analysis
def analyze_results(results):
    # Extract metrics for UCS and A*
    ucs_times = [r['ucs_time'] for r in results]
    ucs_paths = [r['ucs_path_len'] for r in results]
    astar_times = [r['astar_time'] for r in results]
    astar_paths = [r['astar_path_len'] for r in results]

    # Print performance metrics in a table
    print("\nPerformance Analysis:")
    print("| Metric            | UCS Mean | UCS Variance | A* Mean | A* Variance |")
    print("|-------------------|----------|--------------|---------|-------------|")
    print(f"| Time (mins)       | {statistics.mean(ucs_times):.2f}    | {statistics.variance(ucs_times):.2f}         | {statistics.mean(astar_times):.2f}   | {statistics.variance(astar_times):.2f}        |")
    print(f"| Path Length       | {statistics.mean(ucs_paths):.2f}     | {statistics.variance(ucs_paths):.2f}         | {statistics.mean(astar_paths):.2f}    | {statistics.variance(astar_paths):.2f}        |")

    # Theoretical analysis of both algorithms
    print("\nTheoretical Analysis:")
    for algo, times, paths in [('UCS', ucs_times, ucs_paths), ('A*', astar_times, astar_paths)]:
        print(f"{algo}:")
        print(f"  Completeness: Yes (finite 6×6 maze with 36 nodes)")
        print(f"  Optimality: Yes ({'cost-based exploration' if algo == 'UCS' else 'admissible and consistent heuristic'})")
        print(f"  Time Complexity: O(b^d), b ≈ 6, d ≈ {min(paths)}-{max(paths)}")
        print(f"  Space Complexity: O(b^d) due to priority queue storing up to {max(times)} nodes")
        print(f"  Variance Analysis: {'Low variance' if statistics.variance(times) < 1 else 'Higher variance'} ({statistics.variance(times):.2f}) indicates {'stable' if statistics.variance(times) < 1 else 'variable'} performance across mazes")

# Main execution: Run experiments and analyze results
results = run_experiments()
analyze_results(results)


Maze 1
Start: 8, Goal: 26, Barriers: [2, 12, 19, 23]
UCS - Visited: [0, 1, 3, 4, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 18, 20, 21, 22, 26], Time: 19 mins, Path: [8, 14, 20, 26]
UCS Visualization:
00 01 XX 03 04 05
06 07 SS 09 10 11
XX 13 ** 15 16 17
18 XX ** 21 22 XX
24 25 GG 27 28 29
30 31 32 33 34 35
A* - Visited: [8, 14, 20, 26], Time: 4 mins, Path: [8, 14, 20, 26]
A* Visualization:
00 01 XX 03 04 05
06 07 SS 09 10 11
XX 13 ** 15 16 17
18 XX ** 21 22 XX
24 25 GG 27 28 29
30 31 32 33 34 35

Maze 2
Start: 11, Goal: 26, Barriers: [27, 14, 13, 8]
UCS - Visited: [2, 3, 4, 5, 9, 10, 11, 15, 16, 17, 20, 21, 22, 23, 26, 28, 29, 35], Time: 18 mins, Path: [11, 16, 21, 26]
UCS Visualization:
00 01 02 03 04 05
06 07 XX 09 10 SS
12 XX XX 15 ** 17
18 19 20 ** 22 23
24 25 GG XX 28 29
30 31 32 33 34 35
A* - Visited: [10, 11, 16, 17, 21, 26], Time: 6 mins, Path: [11, 16, 21, 26]
A* Visualization:
00 01 02 03 04 05
06 07 XX 09 10 SS
12 XX XX 15 ** 17
18 19 20 ** 22 23
24 25 GG XX 28 29
30 31 32 33 34 